# Entities as Experts

This notebook is a code implementation of the paper "Entities as Experts: Sparse Memory Access with Entity Supervision" by Févry, Baldini Soares, FitzGerald, Choi, Kwiatowski.

## Problem definition and high-level model description

We want to perform question answering on typical one-shot questions that require external knowledge or context. For example, in order to answer the question "Which country was Charles Darwin born in?" one needs some text providing answers on typical structured scenarios.

In this case, however, we want to rely on knowledge-graph extracted information. For example, in the question given here, we can prune out unrelated to the antropologist and evolution theorist Charles Darwins, e.g. Charles River, Darwin City etc. 

In the paper, the authors propose to augment BERT in the task of cloze-type question answering by leveraging an Entity Memory extracted from e.g. a Knoweldge Graph.

![Entity as Experts description](images/eae_highlevel.png)

The Entity Memory is a simple bunch of embeddings of entities extracted from a Knowledge Graph. Relationships are ignored (see the Facts as Experts paper and notebook to see how they could be used).

## Datasets

> We assume access to a corpus $D={(xi,mi)}$,where all entity mentions are detected but not necessarily  all  linked  to  entities.   We  use  English Wikipedia as our corpus, with a vocabulary of 1m entities. Entity links come from hyperlinks, leading to 32m 128 byte contexts containing 17m entity links.

In the appendix B, it is explained that:

> We build our training corpus of contexts paired with entity mention labels from the 2019-04-14 dump of English Wikipedia. We first divide each article into chunks of 500 bytes,resulting in a corpus of 32 million contexts withover 17 million entity mentions. We restrict our-selves  to  the  one  million  most  frequent  entities
(86% of the linked mentions).

Given that the dump 2019-04-14 is not available at the time of writing, we will adopt the revision 2020-11-01.

Entities are thus partially extracted by link annotations (e.g. they associate with each token a mention if that token belongs to a wikipedia url).

## Mention Detection

> In addition to the Wikipedia links, we annotaten each sentence with unlinked mention spans using the mention detector from Section 2.2

The mention detection head discussed in Section 2.2 is a simple BIO sequence: each token is annotated with a B (beginning), I (inside) or O (outside) if they are respectivelly beginning, inside or outside of a mention. The reason why we use both BIO and EL is to avoid inconsistencies.

There is a catch. In the paper, they explain they used Google NLP APIs to perform entity detection and linking on large-scale Wikipedia entries, that is, to have a properly annotated Wikipedia dataset.

Since we cannot technically afford this, we will use spacy's entity detection and linking capabilities as a baseline. Data quality 

## Chunking

- Split articles by chunks of 500 bytes (assuming unicode encoding).
- We will elide sentences till the last period to make sure they reach such limit without giving weird effects.

## Tokenization:

- BERT Tokenizer (e.g. Wordpiece) using lowercase vocabulary, limited to 128 distinct word-piece tokens.

In [1]:
from tools.providers import WikipediaProvider

WikipediaProvider.dump_full_dataset(revision="20201101")

100%|██████████| 255M/255M [00:50<00:00, 5.05MiB/s] 


100%|██████████| 351M/351M [01:08<00:00, 5.12MiB/s] 


100%|██████████| 387M/387M [01:15<00:00, 5.14MiB/s] 


100%|██████████| 4.79M/4.79M [00:01<00:00, 4.32MiB/s]


100%|██████████| 4.68M/4.68M [00:01<00:00, 4.34MiB/s]


...

100%|██████████| 5.02M/5.02M [00:01<00:00, 4.29MiB/s]


100%|██████████| 5.09M/5.09M [00:01<00:00, 4.40MiB/s]


100%|██████████| 5.41M/5.41M [00:01<00:00, 4.45MiB/s]


100%|██████████| 64.9M/64.9M [00:12<00:00, 5.09MiB/s]


100%|██████████| 276M/276M [00:54<00:00, 5.10MiB/s] 


## Model

In the paper, the authors explain they used a modified BERT.

In [3]:
from torch.nn import Module, Embedding, Dropout, ModuleList, Linear
import torch.nn as nn
import torch
import math

GELU = torch.nn.GELU
LayerNorm = torch.nn.LayerNorm

l0 = 4
l1 = 8

class SublayerConnection(Module):
    """A residual connection followed by layer norm and applied dropout.
    """
    
    def __init__(self, size, dropout):
        """
        :param size the size of the layer norm
        :param dropout the dropout rate
        """
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = Dropout(dropout)
    
    def forward(self, x, sublayer):
        return x + self.dropout(self.norm(sublayer(x)))


class TokenEmbedding(Embedding):
    """A token embedding is a mere synonim of the default torch embedder.
    
    It simply keeps a fixed vocabulary size.
    """
    def __init__(self, vocab_size, embed_size=512):
        super().__init__(vocab_size, embed_size, padding_idx=0)
    

class PositionalEmbedding(Module):
    def __init__(self, d_model, max_len=512):
        """
        Setup positional embeddings
        
        :param d_model: the size of a single embedding
        :param max_len: the maximum number of tokens to embed
        """
        super().__init()
        
        # Compute the positional encodings once in log space
        pe = torch.zeros(max_len, d_model).float()
        pe.require_grad = False
        
        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float() * (math.log(10000.0) / d_model)).exp()
         
        # I have no clue here
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        return self.pe[:, :x.size(1)]

    
class SegmentEmbedding(Embedding):
    """
    Segment Embedding. It works in a very similar way to token embed but we constrain the size
    of our embedding to 3 (before, current, next sentence?)
    """
    def __init__(self, embed_size=512):
        super().__init__(3, embed_size, padding_idx=0)
        

class BERTEmbedding(Module):
    """
    BERT Embeddings which is made of the following features:
    
    1. Token Embedding: normal embedding matrix
    2. Positional Embedding: additional position information
    3. SegmentEmbedding: Additional sentence segment info (eg. tok_1:1, tok_2:2)
    """
    def __init__(self, vocab_size, embed_size, dropout=0.1):
        """
        :param vocab_size: the size of the vocabulary
        :embed_size: the vector dimensionality of the embeddings
        :dropout: dropout rate
        """
        
        super().__init__()
        self.token = TokenEmbedding(vocab_size=vocab_size, embed_size=embed_size)
        self.position = PositionalEmbedding(embed_size=self.token.embedding_dim)
        self.segment = SegmentEmbedding(embed_size=self.token.embedding_dim)
        self.dropout = nn.Dropout(p=dropout)
        self.embed_size = embed_size
        
    def forward(self, sequence, segment_label):
        x = self.token(sequence) + self.position(sequence) + self.segment(segment_label)
        return self.dropout(x)


class Attention(nn.Module):
    """Attention mechanism in BERT.
    
    Compute Scaled Dot Product Attention.
    
    :return return p_attn * value, p_attn (possibly with applied dropout)
    """
    def forward(self, query, key, value, mask=None, dropout=None):
        
        # apply key to the query to get attention scores
        scores = torch.matmul(query, key.transpose(-2, -1)) \
            / math.sqrt(query.size(-1)) # the original paper suggests doing this
        
        # Be sure to erase out masked values
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        
        # get a probability distribution
        p_attn = F.softmax(scores, dim=-1)
        
        if dropout is not None:
            p_attn = dropout(p_attn)
        
        return torch.matmul(p_attn, value), p_attn

class MultiHeadedAttention(Module):
    """
    Wrap and combine multiple attention heads
    """
    
    def __init__(self, heads, embedding_size, dropout=0.1):
        """
        :param heads the number of attention heads
        :param embedding_size the size of a token embedding. Must be equal to embedding_per_head * heads,
               where embeddding_per_head
        :param dropout: dropout rate
        """
        assert embedding_size % heads == 0
        
        self.embedding_per_head = embedding_size // heads
        self.heads = heads
        
        # query, key, value
        # Note for myself: ModuleList registers all the modules in autograd
        self.linear_layers = ModuleList([Linear(embedding_size, embedding_size) for _ in range(3)])
        self.output_linear = nn.Linera(d_model, d_model)
        self.attention = Attention()
        
        self.dropout = Dropout(p=dropout)
        
    def forward(self):
        batch_size = query.size(0)
        
        # Step 1: extract query, key, value for each
        pass
class TransformerBlock(Module):
    pass